In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from cuml.svm import SVC
from cuml.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import RobustScaler
import gc


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/tabular-playground-series-nov-2021/sample_submission.csv')
train = pd.read_csv('/kaggle/input/tabular-playground-series-nov-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-nov-2021/test.csv')

In [ ]:
sample_submission.head()

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
columns = test.columns[1:]
columns

In [ ]:
train[columns].describe()

In [ ]:
train.isnull().sum().sum()

In [ ]:
target = train[['target']].values
train.drop(['target', 'id'], axis=1, inplace=True)
test.drop(['id', ], axis=1, inplace=True)

In [ ]:
%%time
train_test = pd.concat([train, test], axis =0)
RS = RobustScaler()
RS.fit(train_test)
del train_test
gc.collect()
gc.collect()
train = RS.transform(train)
test = RS.transform(test)
del RS
gc.collect()
gc.collect()

In [ ]:
train

In [ ]:
#X = train[columns].values
#X_test = test[columns].values
#target = train['target'].values

In [ ]:
train_oof = np.zeros((train.shape[0],))
test_preds = np.zeros((test.shape[0],))
train_oof.shape

In [ ]:
target.dtype

In [ ]:
%%time
n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(train)):
    print("Fitting fold", jj+1)
    train_features = train[train_index]
    train_target = target[train_index]

    
    val_features = train[val_index]
    val_target = target[val_index]

    
    model = SVC(C= 1, probability=True)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)[:,1]
    train_oof[val_index] = val_pred.flatten()
    test_preds += model.predict_proba(test)[:,1]/n_splits
    print(roc_auc_score(val_target,val_pred))

In [ ]:
np.save('train_oof', train_oof)
np.save('test_preds', test_preds)

In [ ]:
sample_submission['target'] = test_preds
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head()